In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
import numpy as np
import pandas as pd
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder

In [2]:
df=pd.read_json('News_Category_Dataset_v3.json',lines=True)
df.head(5)

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [3]:
df.dropna(inplace=True)
df=df.drop(['date','link','authors'],axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   headline           209527 non-null  object
 1   category           209527 non-null  object
 2   short_description  209527 non-null  object
dtypes: object(3)
memory usage: 4.8+ MB


In [5]:
df.isnull().sum()

headline             0
category             0
short_description    0
dtype: int64

In [6]:
df['namecategory']=df['category']

In [7]:
df['short_description']=df['short_description'].apply(lambda x:x.lower())


In [8]:
len(df['category'].unique())

42

In [9]:
# df=df[:20000]

In [10]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [11]:
df['category']

0         U.S. NEWS
1         U.S. NEWS
2            COMEDY
3         PARENTING
4         U.S. NEWS
            ...    
209522         TECH
209523       SPORTS
209524       SPORTS
209525       SPORTS
209526       SPORTS
Name: category, Length: 209527, dtype: object

In [12]:
def stem(text):
    l=[]
    for i in text.split():
        l.append(ps.stem(i))
    return "".join(l)

In [13]:
df['category'].apply(stem)
df.head()

,headline,category,short_description,namecategory
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,health experts said it is too early to predict...,U.S. NEWS
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,he was subdued by passengers and crew when he ...,U.S. NEWS
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""until you have a dog you don't understand wha...",COMEDY
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""accidentally put grown-up toothpaste on my to...",PARENTING
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,amy cooper accused investment firm franklin te...,U.S. NEWS


In [14]:
# label_encoder = LabelEncoder()
# encoded_labels = label_encoder.fit_transform(df['category'])
# df['category'] =encoded_labels
# # decoded_labels = label_encoder.inverse_transform(df['category'])
# # decoded_labels
# label_mapping = {label: original_label for label, original_label in zip(range(len(label_encoder.classes_)), label_encoder.classes_)}
# decoded_labels = [label_mapping[label] for label in encoded_labels]
# decoded_labels

In [15]:
sorted_df=df.sort_values(by='category')  # default ascending=True
sequence=sorted_df['namecategory'].unique()

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df['headline'], df['category'], test_size=0.15, random_state=42)

In [17]:
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

In [18]:
# svm_classifier_bow = SVC()
nb_classifier_bow = MultinomialNB()

# svm_classifier_bow.fit(X_train_bow, y_train)
nb_classifier_bow.fit(X_train_bow, y_train)

MultinomialNB()

In [19]:
# y_pred_svm = svm_classifier_bow.predict(vectorizer.transform(X_test))
y_pred_nb = nb_classifier_bow.predict(vectorizer.transform(X_test))

# print("SVM Results:")
# print(classification_report(y_test, y_pred_svm, zero_division=1))

print("Naive Bayes Results:")
print(classification_report(y_test, y_pred_nb, zero_division=1))

Naive Bayes Results:
                precision    recall  f1-score   support

          ARTS       0.78      0.03      0.06       227
ARTS & CULTURE       0.25      0.00      0.01       208
  BLACK VOICES       0.62      0.17      0.27       702
      BUSINESS       0.58      0.29      0.39       925
       COLLEGE       1.00      0.01      0.01       155
        COMEDY       0.61      0.24      0.34       757
         CRIME       0.49      0.53      0.51       535
CULTURE & ARTS       0.92      0.07      0.13       157
       DIVORCE       0.91      0.40      0.56       502
     EDUCATION       0.50      0.01      0.01       154
 ENTERTAINMENT       0.46      0.80      0.59      2558
   ENVIRONMENT       0.89      0.03      0.07       230
         FIFTY       0.00      0.00      1.00       192
  FOOD & DRINK       0.66      0.64      0.65       965
     GOOD NEWS       0.50      0.02      0.05       208
         GREEN       0.37      0.07      0.12       399
HEALTHY LIVING       0.36 

In [20]:
# y_test
# sequence=df['category'].unique()
# print(sequence[y_pred_nb[0]-1])

In [21]:
y_pred1 = nb_classifier_bow.predict(vectorizer.transform(['''
At its essence, physics seeks to elucidate the nature of matter, energy, space, and time
''']))
y_pred1[0]

'WELLNESS'

In [28]:
import joblib

In [29]:
joblib.dump(vectorizer,'Vectorizer')
joblib.dump(nb_classifier_bow,'NaiveBayes')

['NaiveBayes']

In [22]:
# nltk.download('averaged_perceptron_tagger')
# def pos_tagging(text):
#     words = word_tokenize(text)
#     pos_tags = pos_tag(words)
#     return ' '.join([tag[1] for tag in pos_tags])


# X_train_pos = X_train.apply(pos_tagging)
# X_test_pos = X_test.apply(pos_tagging)

# # Feature Extraction - Bag of Words (BoW) for POS tags
# vectorizer_pos = CountVectorizer()
# X_train_pos_bow = vectorizer_pos.fit_transform(X_train_pos)
# X_test_pos_bow = vectorizer_pos.transform(X_test_pos)

In [23]:
# nltk.download('words')
# nltk.download('maxent_ne_chunker')
# def named_entities(text):
#     words = word_tokenize(text)
#     ner_tags = ne_chunk(pos_tag(words))
#     return ' '.join([tag.label() for tag in ner_tags if isinstance(tag, nltk.Tree)])

# X_train_ner = X_train.apply(named_entities)
# X_test_ner = X_test.apply(named_entities)

# vectorizer_ner = CountVectorizer()
# X_train_ner_bow = vectorizer_pos.fit_transform(X_train_ner)
# X_test_ner_bow = vectorizer_pos.transform(X_test_ner)

In [24]:
# # Train models
# svm_classifier_ner = SVC()
# nb_classifier_ner= MultinomialNB()

# svm_classifier_ner.fit(X_train_ner_bow, y_train)
# nb_classifier_ner.fit(X_train_ner_bow, y_train)

In [25]:
# X_train_mix =np.concatenate([ X_train_ner_bow.toarray()  , X_train_pos_bow.toarray()] , axis = 1)
# X_test_mix = np.concatenate([ X_test_ner_bow.toarray()  , X_test_pos_bow.toarray()] , axis = 1)

In [26]:
# svm_classifier_mix = SVC()
# nb_classifier_mix = MultinomialNB()

# svm_classifier_mix.fit(X_train_mix, y_train)
# nb_classifier_mix.fit(X_train_mix, y_train)

In [27]:
# # Evaluate models
# y_pred_svm = svm_classifier_mix.predict(X_test_mix)
# y_pred_nb = nb_classifier_mix.predict(X_test_mix)

# print("SVM Results:")
# print(classification_report(y_test, y_pred_svm, zero_division=1))

# print("Naive Bayes Results:")
# print(classification_report(y_test, y_pred_nb, zero_division=1))